# Pipeline Integrado SentinelaMSF

Este notebook consome dados diretamente do banco SQLite (`sentinela.db`), alimentado pelos scripts Python da aplicação:

- `src/sentinela/ingest_malaria.py`: ingestão de casos históricos de malária.
- `src/sentinela/ingest_predicoes.py`: ingestão das predições de risco (ML).
- `src/sentinela/ingest_weather.py`: ingestão de alertas climáticos.
- `src/sentinela/ingest_enchentes.py`: ingestão de enchentes detectadas.

As tabelas lidas são:
- `malaria_casos`
- `enchentes_detectadas`
- `alertas_inmet`
- `predicoes_risco`

Isso garante que toda análise e modelagem esteja sempre alinhada com os dados mais recentes e o fluxo real da aplicação.


In [ ]:
import pandas as pd
import sqlite3

# Conexão com o banco
conn = sqlite3.connect('../sentinela.db')  # Ajuste o caminho se necessário

# Leitura das tabelas
malaria_df = pd.read_sql_query('SELECT * FROM malaria_casos', conn)
enchentes_df = pd.read_sql_query('SELECT * FROM enchentes_detectadas', conn)
alertas_df = pd.read_sql_query('SELECT * FROM alertas_inmet', conn)
predicoes_df = pd.read_sql_query('SELECT * FROM predicoes_risco', conn)


In [ ]:
# Conversão de datas
malaria_df['data'] = pd.to_datetime(malaria_df['data'])
enchentes_df['data'] = pd.to_datetime(enchentes_df['data'])
alertas_df['data_alerta'] = pd.to_datetime(alertas_df['data_alerta'])
predicoes_df['data_predicao'] = pd.to_datetime(predicoes_df['data_predicao'])


## Visualização rápida dos dados carregados


In [ ]:
display(malaria_df.head())
display(enchentes_df.head())
display(alertas_df.head())
display(predicoes_df.head())


## Integração dos dados para análise

Aqui unimos os dados de malária, enchentes e predições para análise conjunta por município e data.


In [ ]:
# Exemplo: merge entre malária e enchentes
df = malaria_df.merge(
    enchentes_df[['municipio', 'data', 'intensidade']],
    on=['municipio', 'data'],
    how='left'
)

# Merge com predições
df = df.merge(
    predicoes_df[['municipio', 'data_predicao', 'risco', 'score']],
    left_on=['municipio', 'data'],
    right_on=['municipio', 'data_predicao'],
    how='left'
)

df.drop('data_predicao', axis=1, inplace=True)
df.head()


## Análise Exploratória

Aqui você pode seguir com análise estatística, visualizações, gráficos de evolução temporal, mapas, etc.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Exemplo: evolução dos casos de malária
malaria_por_data = malaria_df.groupby('data')['casos'].sum()
plt.figure(figsize=(10,4))
malaria_por_data.plot()
plt.title('Evolução dos casos de malária')
plt.xlabel('Data')
plt.ylabel('Casos')
plt.show()


## Observações finais

- Este notebook está alinhado com o fluxo real da aplicação.
- Sempre leia os dados diretamente do banco para garantir atualização.
- Para análises espaciais, use as colunas de latitude/longitude se disponíveis nas tabelas.
- Para modelagem ML, siga a integração dos dados já mostrada.
